In [ ]:
using Pkg; Pkg.activate("..")
using SparseArrays, LinearAlgebra, Ferrite, LaTeXStrings
using FerriteViz, WGLMakie, Makie
Makie.inline!(true);

In this notebook we will use the code `Ferrite.jl` to solve our simplest model problem, 
$$\begin{aligned} 
 - \Delta u  &= f, \qquad \Omega, \\ 
          u &= 0, \qquad \partial\Omega
\end{aligned}$$ 
In the assignment we will generalize this to some more intersting problems.

In [ ]:
grid = generate_grid(Triangle, (20, 20))
FerriteViz.wireframe(grid, markersize = 10, strokewidth = 2, 
                     figure = (resolution=(500,500),))

In [ ]:
# here we specify the finite element space and the quadrature rule. 
# ip_geo specifies the space for isoparametric elements i.e. 
# elements that can have curved boundaries. Ferrite forces us 
# to specify this - but we can just ignore it from here on.
dim = 2
ip = Lagrange{dim, RefTetrahedron, 2}()
ip_geo = Lagrange{dim, RefTetrahedron, 1}()
qr = QuadratureRule{dim, RefTetrahedron}(4)
# qr.points and wr.weights ... 

# the next object facilitates the process of evaluating 
# shape functions and their derivatives at quadrature 
# points; it pre-evaluates them in the reference 
# element and then allows us to access those values 
# during the assembly
cellvalues = CellScalarValues(qr, ip, ip_geo)

In [ ]:
# a DOF handler is an object that manages how degrees 
# of freedom of FE functions are stored, e.g. for P1
# this object will assigne DOFs to the element nodes
# for P2 it assigns DOFs to nodes and edges. 
dh = DofHandler(grid)
# here we register a scalar-valued function "u" 
# with the DOF handler. 
add!(dh, :u, 1, ip)
close!(dh)

# Once you know the mesh and the DOFs you can already 
# determine the sparsity pattern of the system matrix
# this is now precomputed for later use. 
K = create_sparsity_pattern(dh)

In [ ]:
# The Dirichlet Boundary condition is handled 
# as a constraint. Typical FEM software frameworks
# usually have objects the manage constraints for us
ch = ConstraintHandler(dh)

# The next line is just a simple trick to get the 
# boundary information - this can be more complicated 
# in general. 
∂Ω = union(getfaceset.(Ref(grid), ["left", "right", "top", "bottom"])...)
# now we are generate a Dirichlet bc constraint ...
dbc = Dirichlet(:u, ∂Ω, (x, t) -> 0)
# ... and register it with the constraint handler 
add!(ch, dbc)
close!(ch)

In [ ]:
# We are now ready to assemble the finite element system. 
# This is normally done in two functions, 
#   (1) assemble_global!  : the outer assembly loop, 
#                           this is just boilerplate 
#   (2) assemble_element! : the assembly work on an individual element, 
#                           here the problem-specific work happens
#
# the ! suffix is a Julia convention to indicate that the functions 
# modify the input argument(s)

function assemble_global!(cellvalues, dh, K, ffun)
   # we pre-allocate the element stiffness matrix and element force vector
   # these will be passed to the element assembly to avoid many allocations
   n_basefuncs = getnbasefunctions(cellvalues)
   Ke = zeros(n_basefuncs, n_basefuncs)
   fe = zeros(n_basefuncs)
    
   # Allocate global force vector f
   f = zeros(ndofs(dh))
    
   # Create an assembler: this object knows how to write 
   # the local arrays Ke, fe into the global arrays K, f 
   assembler = start_assemble(K, f)
    
   # Loop over all cells; this is managed by the DOF handler 
   # since the `cell` comes with information about local 
   # DOFs attached.
   for cell in CellIterator(dh)
       # Reinitialize cellvalues for this cell
       # `cellvalues` has iterators attached that need 
       # to be reset. This seems unnecessary and probably 
       # just a poor code design decision. 
       reinit!(cellvalues, cell)
       # ==========================================
       # Compute element contribution; 
       # this is where the actual work happens
       assemble_element!(Ke, fe, cell, cellvalues, ffun)
       # ==========================================
       # local-to-global assemble Ke and fe into K and f
       assemble!(assembler, celldofs(cell), Ke, fe)
   end
   return K, f
end

function assemble_element!(Ke, fe, cell, cellvalues, ffun)
   # number of local basis functions    
   n_basefuncs = getnbasefunctions(cellvalues)
   # Reset the local arrays 
   fill!(Ke, 0); fill!(fe, 0)
   # precompute the cell coordinates 
   cell_coords = getcoordinates(cell)
    
   # Loop over quadrature points
   for i_q in 1:getnquadpoints(cellvalues)
       # Get the quadrature weight for the current quad point
       # this includes the det(F) terms. It can be thought of 
       # as the volume element (hence dΩ)
       dΩ = getdetJdV(cellvalues, i_q)
        
       # evaluate f at the quadrature point 
       ξ_q = spatial_coordinate(cellvalues, i_q, cell_coords)
       f_q = ffun(ξ_q)
        
       # Loop over test shape functions (basis functions)
       for i in 1:n_basefuncs
           # get the values v = ψ_i(ξ_q), ∇v = ∇ψ_i(ξ_q)
           v  = shape_value(cellvalues, i_q, i)
           ∇v = shape_gradient(cellvalues, i_q, i)
            
           # Add contribution to fe
           fe[i] += f_q * v * dΩ
            
           # Loop over trial shape functions
           for j in 1:n_basefuncs
               ∇u = shape_gradient(cellvalues, i_q, j)
               # Add contribution to Ke
               Ke[i, j] += dot(∇v, ∇u) * dΩ
           end
       end
   end
   return Ke, fe
end


In [ ]:
# we can assemble the system
ffun = ξ -> 5.0
K, f = assemble_global!(cellvalues, dh, K, ffun)

# apply the constraints
apply!(K, f, ch)

# solve the resulting linear system
u = K \ f;


In [ ]:
# visualize the solution (always in a separate cell!)
plotter = FerriteViz.MakiePlotter(dh, u)

# FerriteViz.solutionplot(plotter,field=:u, figure = (resolution = (500,500),))

fig = FerriteViz.surface(plotter, field=:u, 
                   figure = (resolution = (700,700,),))

In [ ]:
# In a way this code looks messier and more difficult 
# to read than our naive P1 code. But the point is that 
# it is much easier to change the finite element method
# for example if we want to change to a different order 
# this is now straightforward: 

function setup_fem(N, p)
    grid = generate_grid(Triangle, (N, N))
    dim = 2
    ip = Lagrange{dim, RefTetrahedron, p}()
    ip_geo = Lagrange{dim, RefTetrahedron, 1}()
    qr = QuadratureRule{dim, RefTetrahedron}(3*p)
    cellvalues = CellScalarValues(qr, ip, ip_geo)
    dh = DofHandler(grid)
    add!(dh, :u, 1, ip)
    close!(dh)
    K = create_sparsity_pattern(dh)
    ch = ConstraintHandler(dh)
    ∂Ω = union(getfaceset.(Ref(grid), ["left", "right", "top", "bottom"])...)
    dbc = Dirichlet(:u, ∂Ω, (x, t) -> 0)
    add!(ch, dbc)
    close!(ch)    
    return cellvalues, dh, ch, K
end

function solve_fem(cellvalues, dh, ch, K, ffun)
    K, f = assemble_global!(cellvalues, dh, K, ξ -> ffun(ξ));
    apply!(K, f, ch)
    u = K \ f;
    return u 
end

In [ ]:
# e.g. with a smaller grid size but cubic elements
cellvalues, dh, ch, K = setup_fem(8, 3)

# solve 
u = solve_fem(cellvalues, dh, ch, K, ξ -> 5.0)

# visualize : we can't see a difference in the solution
# but this is good of course. We will look at errors next.
fig = FerriteViz.surface(plotter, field=:u, 
                   figure = (resolution = (700,700,),))

## Method of Manufactured Solutions

We specify a solution $u(x)$ and compute $f = - \Delta u(x)$ via automatic differentiation. Then we use $f$ as the input into our FEM.

CAREFUL: Sometimes when we do this we accidentally give $u$ more regularity than is natural!! We will return to this soon.

In [ ]:
using ForwardDiff, LinearAlgebra

u_ex(x) = sin(π*x[1]) * sin(π*x[2])
ffun = x -> - tr( ForwardDiff.hessian(u_ex, x) )
;

With the exact solution in hand, we can now compute the errors. And we do it via quadrature! 

In [ ]:
"""
- N : # grid pts in each coordinate direction, h = 1/N
- k : polynomial order of the FEM  (Lagrange element)
- ffun : forcing function
"""
function fem_errors(N, k, ffun)
    cellvalues, dh, ch, K = setup_fem(N, k)
    u = solve_fem(cellvalues, dh, ch, K, ffun)
    return compute_errors(cellvalues, dh, u, u_ex)
end

function compute_errors(cellvalues, dh, u, u_ex)
    n_basefuncs = getnbasefunctions(cellvalues)
    err_L2 = 0.0 
    err_H1 = 0.0 
    
    # loop over cells (= elements)
    for cell in CellIterator(dh)
        reinit!(cellvalues, cell)        
        n_basefuncs = getnbasefunctions(cellvalues)
        cell_coords = getcoordinates(cell)
        
        # we also need the local degrees of freedom 
        u_cell = u[cell.dofs]
    
        vK = 0.0
        for i_q in 1:getnquadpoints(cellvalues)
            dΩ = getdetJdV(cellvalues, i_q)
            ξ_q = spatial_coordinate(cellvalues, i_q, cell_coords)
            u_q   = u_ex(ξ_q)
            ∇u_q  = ForwardDiff.gradient(u_ex, ξ_q)
            uh_q  = function_value(cellvalues, i_q, u_cell)
            ∇uh_q = function_gradient(cellvalues, i_q, u_cell)
            err_L2 += dΩ * (u_q - uh_q)^2
            err_H1 += dΩ * norm(∇u_q - ∇uh_q)^2
        end
    end
    return sqrt(err_L2), sqrt(err_H1)
end

In [ ]:
NN = [4, 8, 16, 32, 64, 128]
errs_L2 = Float64[] 
errs_H1 = Float64[]
for N in NN
    err_L2, err_H1 = fem_errors(N, 1, ffun)
    push!(errs_L2, err_L2)
    push!(errs_H1, err_H1)
end
;

In [ ]:
# ... and visualize them
fig = Figure(size = (400, 400); fontsize=30)
ax = Axis(fig[1, 1], xlabel = L"h^{-1}", ylabel = L"\text{error}", 
          xscale = log10, yscale = log10,
          title = "Error P1-FEM")
scatterlines!(NN, errs_L2; linewidth=5, markersize=20, label=L"L^2")
scatterlines!(NN, errs_H1; linewidth=5, markersize=20, label = L"H^1")
NN1 = NN[3:5]
lines!(NN1, 6 ./ NN1; color=:black, linewidth=3, label = L"h, h^2")
lines!(NN1, 5 ./ NN1.^2; color=:black, linewidth=3)
axislegend(ax)
fig

Equipped with the entire `Ferrite.jl` infrastructure behind us we can now do the same with Pk FEM with no changes to the script.

In [ ]:
k = 3   # try different k
NN = [4, 8, 16, 32, 64]
errs_L2 = Float64[] 
errs_H1 = Float64[]
for N in NN
    err_L2, err_H1 = fem_errors(N, k, ffun)
    push!(errs_L2, err_L2)
    push!(errs_H1, err_H1)
end

# as an exception we break our rule of doing computation 
# and visualization in separate cells. 

fig = Figure(size = (400, 400); fontsize=30)
ax = Axis(fig[1, 1], xlabel = L"h^{-1}", ylabel = L"\text{error}", 
          xscale = log10, yscale = log10,
          title = "Error P$k-FEM")
scatterlines!(NN, errs_L2; linewidth=5, markersize=20, label=L"L^2")
scatterlines!(NN, errs_H1; linewidth=5, markersize=20, label = L"H^1")
NN1 = NN[3:5]
lines!(NN1, 6 ./ NN1.^k; color=:black, linewidth=3, 
       label = latexstring("h^$k, h^$(k+1)"))
lines!(NN1, 5 ./ NN1.^(k+1); color=:black, linewidth=3)
axislegend(ax)
fig

Or it might be interested to plot there errors for different methods on a single plot to compare. 


In [ ]:
KK = 1:5
NN = [ [4, 8, 16, 32, 64, 128], 
       [4, 8, 16, 32, 64, 128], 
       [4, 8, 16, 32, 64, 128], 
       [4, 8, 16, 32, 64, 128], 
       [4, 8, 16, 32, 64], ]
        
errs_L2 = [ Float64[] for _ = 1:length(KK) ] 
errs_H1 = [ Float64[] for _ = 1:length(KK) ]
    
for k in KK, N in NN[k] 
    err_L2, err_H1 = fem_errors(N, k, ffun)
    push!(errs_L2[k], err_L2)
    push!(errs_H1[k], err_H1)
end

In [ ]:
fig = Figure(size = (400, 400); fontsize=30)
ax = Axis(fig[1, 1], xlabel = L"h^{-1}", ylabel = L"|| \nabla u - \nabla u_h ||_{L^2}", 
          xscale = log10, yscale = log10,
          title = L"H^1 \text{Error Pk-FEM}")
for k in KK 
    scatterlines!(NN[k], errs_H1[k]; linewidth=5, markersize=20, 
                  label = latexstring("k = $k"))
    NN1 = (k < 5) ? NN[k][3:5] : NN[k][3:4]
    lines!(NN1, 6/k ./ NN1.^k; color=:black, linewidth=3)
end
axislegend(ax, position = :lb)
fig